# Procesado básico de elecciones (Noviembre 2019)

En este notebook vamos a procesar los datos de las elecciones y vamos a crear un mapa con los ganadores principales. También vamos a analizar el porcentaje de votos en blanco, nulos y abstención. 

Para correr este notebook necesitas haber realizado el preprocesado de datos del notebook 0.

La idea que tengo de visualizar este mapa es dibujar el partido mayoritario. Para eso vamos a colorar cada municipio con el partido más votado (y el segundo más votado si corresponde). Para añadir los datos a mapbox vamos a trabajar con los colores de la siguiente manera: hacemos una condición de color, de modo que el partido más votado aparece su color. Para cambiar los porcentajes de voto, cambiamos el alpha. Por desgracia, luego habrá que cargar todos los datos en mapbox y hacer el estilo por nuestra cuenta, pero es lo que hay.

In [ ]:
import os
import pandas as pd
import numpy as np
import bokeh as bk
import holoviews as hv
from bokeh.models import HoverTool
hv.extension('bokeh')
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

from tqdm import tqdm, tqdm_notebook
from bokeh.models import HoverTool, Panel, Tabs

current_dir = os.path.dirname(os.getcwd())
current_dir

In [ ]:
df_propiedades = pd.read_csv(current_dir + '/datos/info_partidos/info_partidos.txt', sep='\t', quoting=3
                            ).set_index('Siglas')

In [ ]:
tipo = 'congreso'
año = '2019'
mes = '11'

In [ ]:
df = pd.read_pickle(current_dir + f'/datos/{tipo}_{año}_{mes}/resultados_candidaturas.pickle')

Primero vamos a sacar cual es el partido más votado, y con ello vamos a hacer la lista de partidos más votados. Para cada uno de ellos habrá una columna específica que indique su porcentaje de voto por sección.

In [ ]:
df_partidos = df[[i for i in df.columns if i not in ['codigo provincia', 'codigo municipio', 'numero distrito',
       'codigo seccion', 'censo escrutinio', 'votos candidaturas',
       'votos blancos', 'votos nulos', 'codigo', 'PP-FORO']]]
df_partidos = df_partidos.fillna(0)
columnas_partidos = df_partidos.columns

In [ ]:
results = np.unique(np.argmax(df_partidos.values, axis=1), return_counts=True)
counts = results[1]
partidos = [df_partidos.columns[i] for i in results[0]]

df_mas_votados = pd.DataFrame({'partido': [i[:8] for i in partidos], 'nombre':partidos,
                               'numero': counts, 'log': np.log10(counts),
                               'colores': df_propiedades.loc[partidos, 'Color'].values,}
                             ).sort_values(by='numero', ascending=False)

Vemos que hay algunos partidos, como "JF", "ARA-MES-ESQ", o "PUM+J" que, pese a haber sido mayoritarios en algúna sección, su recuento es ínfimo. De hecho, si ploteamos la distribución (abajo), podemos establecer un putno de corte para hacer la selección.

In [ ]:
def hook_generico(p):
    # change just some things about the x-axes
    grey = "#ababab"
    p.xaxis.axis_line_color = grey
    
    # change just some things about the y-axes
    p.yaxis.axis_line_width = 0
    
    # change just some things about the two axes
    for axis in [p.xaxis, p.yaxis]:
        axis.minor_tick_line_width = 0
        axis.major_tick_line_color = grey
        axis.axis_label_text_color  = grey
        axis.major_label_text_color = grey
        axis.major_tick_line_width = 0
    
    p.xaxis.major_label_orientation = 3.14/4
    p.xgrid.visible = False
    p.ygrid.visible = False
    p.outline_line_color = None
    
hover = HoverTool(
    tooltips= """<div><span style="font-size: 17px; font-weight: bold;">@partido</span></div>
        <div><span style="font-size: 15px;">@numero</span></div>""",
    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline'
)

In [ ]:
p1 = figure(x_range=df_mas_votados['partido'],  plot_height=350, plot_width=950, tools='reset,box_zoom', 
            sizing_mode = 'scale_width')
p1.add_tools(hover)
p1.vbar(x='partido', top='numero', width=0.9, color='colores', source=df_mas_votados)
hook_generico(p1)
p1.yaxis.axis_label = "Número de secciones con mayoría"
p1.yaxis.axis_label_text_font_style = 'normal'
tab1 = Panel(child=p1, title="Normal")

p2 = figure(x_range=df_mas_votados['partido'],  plot_height=350, plot_width=950,tools='reset,box_zoom', 
            sizing_mode = 'scale_width')
p2.add_tools(hover)
p2.vbar(x='partido', top='log', width=0.9, color='colores', source=df_mas_votados)
hook_generico(p2)
p2.yaxis.axis_label = "Número de secciones con mayoría (log)"
p2.yaxis.axis_label_text_font_style = 'normal'
tab2 = Panel(child=p2, title="Logarítmico")

tabs = Tabs(tabs=[tab1, tab2])

show(tabs)

## Segunda parte: preparar los dataframes para mapbox
Para preparar los dataframes vamos a dibujar los contornos de manera diferente según el zoom. En el zoom más pequeño se verán las autonomías, y conforme vamos descendiendo iremos mostrando los municipios y las secciones. 
Cada uno de estos dataframes va a incluir la siguiente información:
* Nombre: el nombre que se va a mostrar en pantalla. Para provincia es el nombre de la provincia, para municipio el del municipio, y para la sección el nombre de municipio + dist XX + sec YY.
* Porcentaje_PARTIDO: el porcentaje del partido mayoritario. Para el resto de partidos es 0.
* Porcentaje_1o: El porcentaje del partido más votado.
* Partido_1o: El nombre del partido más votado.
* Partido_2o / Porcentaje_2o
* Partido_3o / Porcentaje_3o
* Porcentaje_blancos / porcentaje_nulos / porcentaje_abstencion

Quedaría sacar el porcentaje del resto de partidos. Estos se calculan directamente en el javascript.

En primer lugar, para calcular el porcentaje de los partidos, vamos a hacerlo para cada dataframe. Asi que los creamos ahora.

In [ ]:
df_secciones = df.copy()
df_municipios = df.copy()
df_provincias = df.copy()

In [ ]:
df_municipios.index = [i[:5] for i in df_municipios.index]
df_provincias.index = [i[:2] for i in df_provincias.index]

In [ ]:
df_municipios = df_municipios.groupby(level=0).sum()
df_provincias = df_provincias.groupby(level=0).sum()

Vemos que hay varios partidos que tienen una minoría de representación. Eliminamos aquellos que aparezcan en menos de 10 secciones y repetimos el contaje. 

In [ ]:
partidos_seleccionados = df_mas_votados['nombre'][df_mas_votados['numero'] > 10].values

In [ ]:
for col in partidos_seleccionados:
    for df_i in [df_secciones, df_municipios, df_provincias]:
        df_i['porcentaje_{}'.format(col)] = 100 * df_i[col] / df_i['votos candidaturas']

Ahora, iterando por fila, vamos a hacer que el resto de partidos menos votados sea 0

In [ ]:
for df_i in [df_provincias, df_municipios, df_secciones]:
    df_i['porcentaje_{}'.format(col)] = 100 * df_i[col] / df_i['votos candidaturas']
    
    porcentaje_1, partido_1, porcentaje_2, partido_2, porcentaje_3, partido_3 = [], [], [], [], [], []
    porcentajes_seleccionados = []
    for row in tqdm(df_i.index):
        df_row = df_i.loc[row, partidos_seleccionados]
        df_row = df_row.sort_values(ascending=False)
        porcentaje_1.append(100 * df_row.iloc[0] / df_i.loc[row, 'votos candidaturas'])
        partido_1.append(df_row.index[0])
        porcentaje_2.append(100 * df_row.iloc[1] / df_i.loc[row, 'votos candidaturas'])
        partido_2.append(df_row.index[1])
        porcentaje_3.append(100 * df_row.iloc[2] / df_i.loc[row, 'votos candidaturas'])
        partido_3.append(df_row.index[2])
        
        list_cols = ['porcentaje_{}'.format(col) for col in partidos_seleccionados if col != df_row.index[0]]  
        
        df_i.loc[row,list_cols] = 0
    
    df_i['porcentaje_1'], df_i['partido_1'] = porcentaje_1, partido_1
    df_i['porcentaje_2'], df_i['partido_2'] = porcentaje_2, partido_2
    df_i['porcentaje_3'], df_i['partido_3'] = porcentaje_3, partido_3
    df_i['porcentaje_blancos'] = 100 * df_i['votos blancos'] /  df_i['censo escrutinio']
    df_i['porcentaje_nulos'] = 100 * df_i['votos nulos'] /  df_i['censo escrutinio']
    df_i['porcentaje_abstencion'] = 100 * (1 - df_i['votos candidaturas'] /  df_i['censo escrutinio'])


Ahora cargamos el mapa. Para ello vamos cargamos el GeoJSON con geopandas, y añadimos las columnas correspondientes.

In [ ]:
import geopandas as gpd

secciones_reciente = gpd.read_file(current_dir + f'/datos/mapas_elecciones/secciones_{año}.json', encoding='utf-8')
municipios_reciente = gpd.read_file(current_dir + f'/datos/mapas_elecciones/municipios_{año}.json', encoding='utf-8')
provincias_reciente = gpd.read_file(current_dir + f'/datos/mapas_elecciones/provincias_{año}.json', encoding='utf-8')

In [ ]:
secciones_reciente = secciones_reciente.set_index('CUSEC', drop=False)
municipios_reciente = municipios_reciente.set_index('CUMUN', drop=False)
provincias_reciente = provincias_reciente.set_index('CPRO', drop=False)

Ya hemos aunado los índices, así que ahora los subimos, y listo.

In [ ]:
combinado_secciones = secciones_reciente.join(df_secciones, how='inner')
combinado_municipios = municipios_reciente.join(df_municipios, how='inner')
combinado_provincias = provincias_reciente.join(df_provincias, how='inner')

In [ ]:
nombre_secciones = []
for row in tqdm_notebook(range(len(combinado_secciones))):
    nombre_secciones.append('{}, dist {}, sec {}'.format(combinado_secciones['NMUN'].iloc[row], 
                                                         combinado_secciones.index[row][-5:-3], 
                                                         combinado_secciones.index[row][-3:], ))

In [ ]:
combinado_secciones['Nombre'] = nombre_secciones
combinado_municipios['Nombre'] = combinado_municipios['NMUN']
combinado_provincias['Nombre'] = combinado_provincias['NPRO']

In [ ]:
if not os.path.exists(current_dir + '/datos/mapas_elecciones/'): os.mkdir(current_dir + '/datos/mapas_elecciones/')
combinado_secciones.to_file(current_dir + f'/datos/mapas_elecciones/porcentaje_mas_votado_{año}_{mes}_secciones.json', driver='GeoJSON', encoding='utf-8')
combinado_municipios.to_file(current_dir + f'/datos/mapas_elecciones/porcentaje_mas_votado_{año}_{mes}_municipios.json', driver='GeoJSON', encoding='utf-8')
combinado_provincias.to_file(current_dir + f'/datos/mapas_elecciones/porcentaje_mas_votado_{año}_{mes}_provincias.json', driver='GeoJSON', encoding='utf-8')

Por último, se sube el archivo json que se ha creade a ``tilesets`` en mapbox. Si por alguna razón falla, cargar en mapshaper y descargar de nuevo.